In [1]:
import pandas as pd
import numpy as np
import random
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `

In [2]:
all_df= pd.read_csv('../../final/data/data_correlated_feats.csv')
#df = df.fillna(-1)
all_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.875670,161.104070,3.981397,NaN,358.191297,77.370085,NaN,4.604753,0,0.508604,86.888802,NaN,NaN,NaN,NaN,35.627010,NaN,Anemia of chronic disease
1,9.296779,457.149834,1.874231,5.544915,NaN,78.717345,NaN,3.543099,0,0.784442,37.659905,NaN,NaN,NaN,NaN,27.890337,NaN,Anemia of chronic disease
2,9.780377,483.216550,4.041312,5.515052,304.155693,100.750651,118.428782,2.912252,1,1.129411,140.917968,47.714319,62.722981,20.039071,46.973234,29.341132,38.936895,Vitamin B12/Folate deficiency anemia
3,9.131688,57.578776,5.778568,1.976049,207.684776,96.855220,NaN,2.828455,0,1.569724,26.393979,NaN,NaN,NaN,NaN,27.395064,NaN,Hemolytic anemia
4,11.395680,NaN,NaN,1.320481,318.250045,102.206351,NaN,3.344904,0,NaN,NaN,NaN,NaN,NaN,NaN,34.187041,NaN,Vitamin B12/Folate deficiency anemia


In [3]:
len(all_df)

70000

In [4]:
class_dict = constants.CLASS_DICT
all_df['label'] = all_df['label'].replace(class_dict)
X = all_df.iloc[:, 0:-1]
y = all_df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train_df = pd.concat([X_train, y_train], axis=1)
X_test_df = pd.concat([X_test, y_test], axis=1)
X_test_df = X_test_df.reset_index(drop=True)
X_test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.116363,NaN,3.781573,2.738413,NaN,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,NaN,5
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,NaN,32.705481,NaN,43.520272,24.375961,142.815207,1
2,11.309450,38.324563,NaN,NaN,455.077909,76.402602,NaN,4.440732,0,NaN,NaN,NaN,NaN,NaN,NaN,33.928350,NaN,4
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071,0
4,11.464002,NaN,NaN,NaN,320.964653,104.287127,NaN,3.297819,0,1.163516,121.616315,105.895897,NaN,9.337462,NaN,34.392007,NaN,7


In [5]:
def insert_nans(df, col, frac):
    non_null_indices = df[df[col].notna()].index.tolist() #getting not null indices
    to_change_indices = random.sample(non_null_indices, int(frac*len(non_null_indices)))
    df.loc[to_change_indices, col] = np.nan
    return df

In [6]:
frac = 0.5

In [7]:
for column in X_train_df.columns:
    if column not in ['hemoglobin', 'gender', 'label']:
        X_train_df = insert_nans(X_train_df, column, frac)
X_train_df.isna().sum()

hemoglobin                   0
ferritin                 39475
ret_count                35840
segmented_neutrophils    35124
tibc                     39018
mcv                      30011
serum_iron               43399
rbc                      30011
gender                       0
creatinine               35811
cholestrol               35811
copper                   45937
ethanol                  43399
folate                   45937
glucose                  43399
hematocrit               28000
tsat                     48402
label                        0
dtype: int64

In [8]:
X_train_df.shape

(56000, 18)

In [9]:
utils.get_dt_performance(X_train_df, 'numeric')

(0.596875,
 0.5794435487408962,
 0.7604318855723976,
 datetime.timedelta(microseconds=1992))

In [10]:
X_train_df.to_csv('../../final/data/train_set_missing_5.csv', index=False)